In [1]:
!pip install beautifulsoup4 requests pandas

In [47]:
import urllib
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pickle
import pandas as pd

In [3]:
def get_cna_articles():
    CNA_URL = "https://www.channelnewsasia.com/"
    # get cna main page

    page = requests.get(CNA_URL)
    soup = BeautifulSoup(page.text, "html.parser")
    # get links to different news articles (the class is for this url, will differ across websites)
    links = soup.find_all('a', attrs={'class': 'h6__link h6__link-- list-object__heading-link'})
    articles = []
    for link in links:
        article_page = requests.get(f"{CNA_URL}{link['href']}")
        article = BeautifulSoup(article_page.text, "html.parser")
        article_text = article.find_all('div', attrs={'class': 'text-long'})
        articles.append((' '.join([[' '.join([child.text for child in div.findChildren('p')])][0] for div in article_text]),
                                     f"{CNA_URL}{link['href']}"))
    
    return articles

In [4]:
# run this block to get saved data/scrape data
CNA_ARTICLES_ADDRESS = Path("cna_articles.pickle")
if CNA_ARTICLES_ADDRESS.is_file():
    with open(CNA_ARTICLES_ADDRESS, "rb") as f:
        cna_articles = pickle.load(f)

else:
    cna_articles = get_cna_articles()
    CNA_ARTICLES_ADDRESS.touch(exist_ok=True)
    with open(CNA_ARTICLES_ADDRESS, "wb") as f:
        pickle.dump(cna_articles, f)

In [5]:
CNA_ARTICLES_ADDRESS.is_file()

True

In [6]:
# scrape stackoverflow
from IPython.core.display import display, HTML
def get_stack_overflow_articles():
    STACK_OVERFLOW_URL = "https://stackoverflow.com/"
    page = requests.get(f"{STACK_OVERFLOW_URL}questions?tab=Frequent")
    soup = BeautifulSoup(page.text, "html.parser")
    # get links to different news articles (the class is for this url, will differ across websites)
    links = soup.find_all("a", "question-hyperlink")
    posts = []
    for idx, link in enumerate(links,1):
        article_site = requests.get(f"{STACK_OVERFLOW_URL}{link['href']}")
        article_page = BeautifulSoup(article_site.text, "html.parser")
        parent = article_page.find("div", "s-prose js-post-body")
        p_tags = parent.findChildren("p")
        post = []
        for p_tag in p_tags:
            post.append(p_tag.text)
            display(HTML(str(p_tag)))
        posts.append((" ".join(post), f"{STACK_OVERFLOW_URL}{link['href']}"))
        if idx == 50: break
    return posts

In [7]:
# run this block to get saved data/scrape data
STACK_OVERFLOW_ARTICLES_ADDRESS = Path("stack_overflow_articles.pickle")
if STACK_OVERFLOW_ARTICLES_ADDRESS.is_file():
    with open(STACK_OVERFLOW_ARTICLES_ADDRESS, "rb") as f:
        stack_overflow_articles = pickle.load(f)

else:
    stack_overflow_articles = get_stack_overflow_articles()
    STACK_OVERFLOW_ARTICLES_ADDRESS.touch(exist_ok=True)
    with open(STACK_OVERFLOW_ARTICLES_ADDRESS, "wb") as f:
        pickle.dump(stack_overflow_articles, f)

In [8]:
stack_overflow_articles

[('When discussing performance with colleagues, teaching, sending a bug report or searching for guidance on mailing lists and here on Stack\xa0Overflow, a reproducible example is often asked and always helpful. What are your tips for creating an excellent example? How do you paste data structures from r in a text format? What other information should you include? Are there other tricks in addition to using dput(), dump() or structure()?  When should you include library() or require() statements?  Which reserved words should one avoid, in addition to c, df, data, etc.? How does one make a great r reproducible example?',
  'https://stackoverflow.com//questions/5963269/how-to-make-a-great-r-reproducible-example'),
 ('What are Null Pointer Exceptions (java.lang.NullPointerException) and what causes them? What methods/tools can be used to determine the cause so that you stop the exception from causing the program to terminate prematurely?',
  'https://stackoverflow.com//questions/218384/wha

In [9]:
# scrape hardwarezone
def get_hwz_articles():
    HWZ_URL = "https://forums.hardwarezone.com.sg/"
    page = requests.get(HWZ_URL)
    soup = BeautifulSoup(page.text, "html.parser")
    # get links to different news articles (the class is for this url, will differ across websites)
    divs = soup.find_all("div", "structItem-title")
    content = []
    for div in divs:
        forum = BeautifulSoup(requests.get(f"{HWZ_URL}{div.find('a')['href']}").text, "html.parser")
#         parent = article_page.find("div", "s-prose js-post-body")
        posts = [comment.text for comment in forum.find_all("div", "bbWrapper")]
        content.append((posts, f"{HWZ_URL}{div.find('a')['href']}"))
    return content

In [10]:
# run this block to get saved data/scrape data
HWZ_ARTICLES_PATH = Path("hwz_articles.pickle")
if HWZ_ARTICLES_PATH.is_file():
    with open(HWZ_ARTICLES_PATH, "rb") as f:
        hwz_articles = pickle.load(f)

else:
    hwz_articles = get_hwz_articles()
    HWZ_ARTICLES_PATH.touch(exist_ok=True)
    with open(HWZ_ARTICLES_PATH, "wb") as f:
        pickle.dump(hwz_articles, f)

In [11]:
# dicussion of writing styles
import random
random.seed(200)

cna_sample = random.sample(cna_articles, 2)
stackoverflow_sample = random.sample(stack_overflow_articles, 2)
hwz_sample = random.sample(hwz_articles, 2)


In [12]:
# is the first word in a sentence capitalized; do sentences follow good grammars; are the proper
# nouns capitalized;

for samples in [cna_sample, stackoverflow_sample, hwz_sample]:
    (sample1, _), (sample2, _) = samples
    print(f"{sample1}\n\n{sample2}\n\n")


MELBOURNE :     Australia have renewed belief after finishing runner-up in the Rugby Championship on the back of four straight wins but they must keep "emptying the tank" on the field to get the country behind them, coach Dave Rennie said. The Wallabies recovered from a 3-0 whitewash by New Zealand to beat South Africa and Argentina twice each in the Rugby Championship and bring momentum into their northern hemisphere tour for tests against Japan, England, Scotland and Wales. "You win a couple of tests and things change a bit but we also know things can swing the other way," Rennie told Australian broadcaster Channel Nine. "We honestly believe if we want to get the country behind us we need consistent high-quality performances and they want to see our boys emptying the tank week after week and if they do that people will respect us regardless of the result. "We've made a start but it's something that's a big focus for us." Rennie, in his second year in charge, said the Wallabies had ma

# Writing Style 

In [48]:
# Apply language_tool package to check for language errors
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

In [ ]:
def getArticleLength(articles):
    lengthOfArticle = 0
    for idx in range(len(articles)):
        text = articles[idx][0]
        tokens = tokenizer.tokenize(text)
        lengthOfArticle += len(tokens)
    return lengthOfArticle

In [49]:
# def checkError(articles):
#     error_dict = {}
#     for idx in range(len(articles)):
#         errors = tool.check(articles[idx][0])
#         for error in errors:
#             if error.category in error_dict:
#                 error_dict[error.category] += 1
#             else:
#                 error_dict[error.category] = 1    
        
#     return error_dict
        

In [144]:
def checkError(articles):
    error_dict = {}
    column_names_correction = ["message", 'context',"suggested_correction", 'issue_type']
    correction_df = pd.DataFrame(columns = column_names_correction)

    for idx in range(len(articles)):
        errors = tool.check(articles[idx][0])
        for error in errors:
            if error.category in error_dict:
                error_dict[error.category] += 1
            else:
                error_dict[error.category] = 1    
        
        
        
            correction_df = correction_df.append({"message": error.message, 
                                                  'context': error.context,
                                                  "suggested_correction": error.replacements, 
                                                  'issue_type':error.ruleIssueType}, ignore_index=True)
        
    return error_dict, correction_df    

## CNA Article Writing Style

In [145]:
cna_errors, cna_correction = checkError(cna_sample)

In [146]:
cna_correction

,message,context,suggested_correction,issue_type
0,Possible typo: you repeated a whitespace,MELBOURNE : Australia have renewed belief ...,[ ],whitespace
1,Use a comma before ‘but’ if it connects two in...,...onship on the back of four straight wins bu...,"[, but]",typographical
2,Possible spelling mistake found.,"...get the country behind them, coach Dave Ren...","[Ronnie, Jennie, Bennie, Rennin, Lennie]",misspelling
3,"If the term is a proper noun, use initial capi...",...mpionship and bring momentum into their nor...,[Northern Hemisphere],misspelling
4,Use a comma before ‘but’ if it connects two in...,... couple of tests and things change a bit bu...,"[, but]",typographical
5,Possible spelling mistake found.,"...o know things can swing the other way,"" Ren...","[Ronnie, Jennie, Bennie, Rennin, Lennie]",misspelling
6,Consider adding a comma here.,...ve if we want to get the country behind us ...,"[us,]",typographical
7,Use a comma before ‘and’ if it connects two in...,...eed consistent high-quality performances an...,"[, and]",typographical
8,Use a comma before ‘but’ if it connects two in...,"...dless of the result. ""We've made a start bu...","[, but]",typographical
9,Possible spelling mistake found.,"...s something that's a big focus for us."" Ren...","[Ronnie, Jennie, Bennie, Rennin, Lennie]",misspelling


In [147]:
cna_errors

{'TYPOGRAPHY': 9,
 'PUNCTUATION': 6,
 'TYPOS': 7,
 'CASING': 2,
 'GRAMMAR': 2,
 'STYLE': 1}

In [148]:
cna_length = getArticleLength(cna_sample)
cna_length

626

## Stack Overflow Article Writing Style

In [80]:
stack_errors, stack_correction = checkError1(stackoverflow_sample)

In [82]:
stack_errors

{'TYPOS': 5, 'PUNCTUATION': 1, 'TYPOGRAPHY': 1}

In [83]:
stack_correction

,message,context,suggested_correction,issue_type
0,Possible spelling mistake found.,"I am developing an application, and everytime ...",[every time],misspelling
1,Possible spelling mistake found.,"...n it, I get the message: Unfortunately, MyA...","[Map, Maps, MPP, Knapp, Lapp, App, Yap, Yaps, ...",misspelling
2,Consider using an m-dash if you do not want to...,...I do to solve this? About this question - o...,[—],typographical
3,Possible spelling mistake found.,"In the official W3c webdirver documentation, i...",[WebDriver],misspelling
4,Possible typo: you repeated a whitespace,... is in the new spec document. However... I...,[ ],whitespace
5,Possible spelling mistake found.,...n some tests on the latest Chrome's own Web...,"[WebDriver, Web driver, Cabdriver]",misspelling
6,Possible spelling mistake found.,... would only ever implement the official Web...,"[WebDriver, Web driver, Cabdriver]",misspelling


In [21]:
stack_length = getArticleLength(stackoverflow_sample)
print(stack_length)

191


## HWZ Article Writing Style

In [131]:
def checkErrorForum(articles):
    error_dict = {}
    column_names_correction = ["message", 'context',"suggested_correction", 'issue_type']
    correction_df = pd.DataFrame(columns = column_names_correction)

    for idx in range(len(articles)):
        for idx1 in range(len(articles[idx][0])):
            errors = tool.check(articles[idx][0][idx1])
            for error in errors:
                if error.category in error_dict:
                    error_dict[error.category] += 1
                else:
                    error_dict[error.category] = 1    
        
        
        
                correction_df = correction_df.append({"message": error.message, 
                                                  'context': error.context,
                                                  "suggested_correction": error.replacements, 
                                                  'issue_type':error.ruleIssueType}, ignore_index=True)
        
    return error_dict, correction_df    

In [133]:
hwz_dict, hwz_correction = checkErrorForum(hwz_sample)

In [134]:
hwz_dict

{'TYPOS': 76, 'CASING': 17, 'TYPOGRAPHY': 1, 'GRAMMAR': 2, 'STYLE': 1}

In [135]:
hwz_correction

,message,context,suggested_correction,issue_type
0,Possible spelling mistake found.,The news anchor got those sibeh act smart kind...,"[side, size, site, sides, sites, sixth, sized,...",misspelling
1,Possible spelling mistake found.,... got those sibeh act smart kind of bin. Sib...,"[Side, Size, Site, Sides, Sites, Sixth, Sized,...",misspelling
2,Possible spelling mistake found.,...e sibeh act smart kind of bin. Sibeh er xin...,"[in, win, bin, pin, tin, min, Lin, din, gin, k...",misspelling
3,Possible spelling mistake found,...art kind of bin. Sibeh er xin to watch. Whe...,[Where's],misspelling
4,Possible spelling mistake found.,...d of bin. Sibeh er xin to watch. Wheres jul...,"[Julie, Julia, Juliet, Julio, Julies, Julien, ...",misspelling
...,...,...,...,...
92,This sentence does not start with an uppercase...,asymmetric said: \t\t\tinsurance agents l...,[Asymmetric],typographical
93,Possible spelling mistake found.,...\t\t Click to expand... I want to the GE C...,"[Change, Chang, Chang i]",misspelling
94,Possible spelling mistake found.,...to terminate my investment link policy. Wah...,"[Was, Way, War, Wei, Wash, Wax, Weigh, WAC, Ah...",misspelling
95,Possible spelling mistake found.,...e impressive cars. The current one from AIA...,"[AIR, VIA, AID, ASIA, CIA, AKA, APA, ANA, ARIA...",misspelling
